In [1]:
import functools
from vnstock3 import Vnstock
import json
import numpy as np
import pandas as pd
from langchain_core.runnables import RunnablePassthrough

from langchain_core.messages import AIMessage, ToolMessage, HumanMessage
from langchain_community.agent_toolkits import create_sql_agent
# from autogen import RAGAgent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
# from langchain.retrievers import ChromaRetriever
# from langchain.embeddings import ChromaEmbedder
from langchain import LLMChain
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
import getpass
import os
from langchain.sql_database import SQLDatabase
# from langgraph.prebuilt import ToolNode, StateGraph, AgentState
from langgraph.graph import END, StateGraph

from typing import Literal
from datetime import datetime, timedelta

In [2]:
stock = Vnstock().stock(symbol='A32', source='TCBS')
df = stock.quote.history(start='2023-1-1', 
                                     end='2024-1-31', 
                                     interval='year')

In [3]:
df['month'] = df['time'].dt.month
df['year'] = df['time'].dt.year
df['time'] = df['time'].dt.to_period('M')
df.set_index('time', inplace=True)
df.groupby(['time']).mean()

,open,high,low,close,volume,month,year
time,,,,,,,
2023-02,26.572500,26.572500,26.465000,26.480000,337.500000,2.0,2023.0
2023-03,26.511739,26.511739,26.511739,26.511739,160.869565,3.0,2023.0
2023-04,25.820000,25.820000,25.820000,25.820000,145.000000,4.0,2023.0
2023-05,27.216500,27.272500,27.040500,27.179500,242.250000,5.0,2023.0
2023-06,26.252727,26.272273,26.213636,26.241818,110.727273,6.0,2023.0
2023-07,26.193333,26.291905,26.193333,26.240000,303.047619,7.0,2023.0
2023-08,27.083077,27.142308,27.062308,27.073077,459.192308,8.0,2023.0
2023-09,29.638947,29.810000,29.472632,29.694737,127.210526,9.0,2023.0
2023-10,26.933182,26.978636,26.933182,26.967273,88.545455,10.0,2023.0


In [11]:
def get_api_portfolio(symbols, n):
    """
    Lấy dữ liệu về một danh mục đầu tư gồm nhiều mã cổ phiếu khác nhau trong một khoảng thời gian

    Args:
        symbols: Một danh sách các mã cổ phiếu trong danh mục đầu tư
    """
    endday = datetime.now()
    startday = endday - timedelta(days=365)
    
    data = pd.DataFrame()
    for symbol in symbols:
        stock = Vnstock().stock(symbol=symbol, source='TCBS')
        try:
            df = stock.quote.history(start=startday.strftime('%Y-%m-%d'), 
                                     end=endday.strftime('%Y-%m-%d'), 
                                     interval='1D')
            if df.empty:
                return {"error": "No data found for symbol {}".format(symbol)}
            
            df['time'] = pd.to_datetime(df['time'])
            df = df.sort_values(by='time', ascending=False)
            
            df['month'] = df['time'].dt.month
            df['year'] = df['time'].dt.year
            
            df['idx_time'] = df['time'].dt.to_period('M')
                        
            df.set_index('idx_time', inplace=True)          
            df = df[['close']]
            df.columns = [symbol]
            data = pd.concat([data, df], axis=1)
        except Exception as e:
            return {"error": "Error for symbol {}: {}".format(symbol, str(e))}
    if data.empty:
        return {"error": "No data found for the given symbols and range"}
    
    data = data.groupby(data.index).mean()
    print(data)
    rets = np.log(data / data.shift(1)).dropna()
    
    result = rets.reset_index().to_dict(orient='records')
    
    return json.dumps(result, default=str)

In [13]:
check=get_api_portfolio(['AAA','A32'],n=6)
check

                AAA        A32
idx_time                      
2023-08   10.783333  27.180000
2023-09   10.242632  29.694737
2023-10    8.856364  26.967273
2023-11    9.055455  27.983182
2023-12    9.379524  28.865238
2024-01    9.715909  31.123636
2024-02   10.943750  45.367500
2024-03   10.904762  34.430000
2024-04   10.438421  37.516316
2024-05   11.152273  31.590000
2024-06   11.460000  33.198000
2024-07   11.728261  33.066522
2024-08   10.550000  35.040500


'[{"idx_time": "2023-09", "AAA": -0.05144315555216674, "A32": 0.0884884106784579}, {"idx_time": "2023-10", "AAA": -0.14542232114660703, "A32": -0.0963458098223358}, {"idx_time": "2023-11", "AAA": 0.022231032778677464, "A32": 0.03697967078334708}, {"idx_time": "2023-12", "AAA": 0.03516170679547651, "A32": 0.031034356695593013}, {"idx_time": "2024-01", "AAA": 0.03523565930437369, "A32": 0.0753295051569786}, {"idx_time": "2024-02", "AAA": 0.11900386259347105, "A32": 0.3768284474520035}, {"idx_time": "2024-03", "AAA": -0.003568951221208245, "A32": -0.2758677123166095}, {"idx_time": "2024-04", "AAA": -0.043706234983660705, "A32": 0.08584764872201436}, {"idx_time": "2024-05", "AAA": 0.06614997833044046, "A32": -0.17193531125845293}, {"idx_time": "2024-06", "AAA": 0.027219402108996846, "A32": 0.04964901833003382}, {"idx_time": "2024-07", "AAA": 0.023138676922500895, "A32": -0.003968290420115657}, {"idx_time": "2024-08", "AAA": -0.1058755282870188, "A32": 0.05798319264886414}]'

In [15]:
def predict_future_prices(api_portfolio,symbols, n):
    """
    Dự đoán giá danh mục đầu tư trong n tháng tới dựa trên dữ liệu đầu vào. Phù hợp cho việc dự báo tình hình của danh mục 

    Args:
        api_portfolio: dữ liệu về một danh mục đầu tư gồm nhiều mã cổ phiếu khác nhau trong một khoảng thời gian
        symbols: Danh sách các mã cổ phiếu khác nhau thuộc danh mục đầu tư đó
        n: Số tháng từ ngày hiện tại để dự đoán.
    """
    data = api_portfolio

    try:
        df = pd.DataFrame(eval(data))
    except (ValueError, SyntaxError) as e:
        print(f"Error creating DataFrame: {e}")
        return
    
    if df.empty or not {'idx_time', 'AAA', 'A32'}.issubset(df.columns):
        print("Data không hợp lệ. Đảm bảo dữ liệu chứa các cột 'idx_time', 'AAA', và 'A32'.")
        return
    
    df['idx_time'] = pd.to_datetime(df['idx_time'] + '-01', format='%Y-%m-%d')
    
    df['days'] = (df['idx_time'] - df['idx_time'].min()).dt.days
    
    df = df.dropna(subset=symbols)
    
    degree = 2
    coeffs = {}

    for symbol in symbols:
        coeffs[symbol] = np.polyfit(df['days'], df[symbol], degree)
    
    future_dates = pd.date_range(start=pd.to_datetime('today').replace(day=1), periods=n, freq='MS')
    future_days = (future_dates - df['idx_time'].min()).days
    
    predictions = {}
    for symbol in symbols:
        predictions[symbol] = np.polyval(coeffs[symbol], future_days)
    
    future_df = pd.DataFrame({
        'idx_time': future_dates.strftime('%Y-%m'),  
        **predictions
    })
    
    last_prediction = future_df.iloc[-1]
    
    result = {symbol: last_prediction[symbol] for symbol in symbols}
    
    return json.dumps(result, default=str)

In [18]:
returns = predict_future_prices(check,['AAA','A32'],6)

In [19]:
import functools
from vnstock3 import Vnstock
import json
import numpy as np
import pandas as pd
from langchain_core.runnables import RunnablePassthrough
# from models.lstm_stock_price import model
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage
from langchain_community.agent_toolkits import create_sql_agent
# from autogen import RAGAgent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
# from langchain.retrievers import ChromaRetriever
# from langchain.embeddings import ChromaEmbedder
from langchain import LLMChain
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
import getpass
import os
from dotenv import load_dotenv, find_dotenv
from langchain.sql_database import SQLDatabase
from langgraph.prebuilt import ToolNode
from typing import Literal, List
from datetime import datetime, timedelta
from scipy.optimize import minimize
load_dotenv(find_dotenv())

True

In [26]:
def calculate_portfolio_return(returns: List[float], weights: List[float]) -> float:
    """
    Tính toán lợi nhuận hàng năm của danh mục đầu tư.

    Args:
        returns: Danh sách lợi nhuận lịch sử của các tài sản trong danh mục đầu tư.
        weights: Danh sách tỷ trọng đại diện cho phần trăm của từng tài sản trong danh mục.
    """
    return np.dot(returns.mean(), weights) * 252

def calculate_portfolio_volatility(returns: List[float], weights: List[float]) -> float:
    """
    Tính toán biến động hàng năm của danh mục đầu tư.

    Args:
        returns:  Danh sách lợi nhuận lịch sử của các tài sản trong danh mục đầu tư.
        weights:  Danh sách tỷ trọng đại diện cho phần trăm của từng tài sản trong danh mục.
    """
    return np.dot(weights, np.dot(returns.cov() * 252, weights)) ** 0.5

def calculate_sharpe_ratio(returns: List[float], weights: List[float], risk_free_rate: float) -> float:
    """
    Tính toán tỷ lệ Sharpe của danh mục đầu tư.

    Args:
        returns: Danh sách lợi nhuận lịch sử của các tài sản trong danh mục đầu tư.
        weights: Danh sách tỷ trọng đại diện cho phần trăm của từng tài sản trong danh mục.
        risk_free_rate: Tỷ lệ lợi nhuận không rủi ro.
    """
    port_return = calculate_portfolio_return(returns, weights)
    port_volatility = calculate_portfolio_volatility(returns, weights)
    sharpe_ratio = (port_return - risk_free_rate) / port_volatility
    return sharpe_ratio
def portfolio_optimize(returns, sharpe_ratio_or_variance=True):
    """
    Tối ưu hóa danh mục đầu tư dựa trên tối ưu lợi nhuận hoặc giảm thiểu rủi ro sau khi biết được tình hình doanh mục đầu tư được dự đoán trong tương lai trước đó.

    Args:
        returns: Thông tin tình hình danh mục đầu tư sau khi dùng hàm predict_future_prices .
        sharpe_ratio_or_variance: Một giá trị boolean chỉ định có tối ưu hóa cho tỷ lệ Sharpe (True) hay biến động (False).
    """
    returns = json.loads(returns)
    returns = pd.DataFrame([returns])
    # returns=returns[returns.columns[1:]]
    # print(returns)
    num_assets = len(returns.columns)
    # print('haha',num_assets)
    init_guess = num_assets * [1. / num_assets]
    bounds = tuple((0, 1) for asset in range(num_assets))
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    if sharpe_ratio_or_variance:
        result = minimize(lambda x: -calculate_sharpe_ratio(returns,x,risk_free_rate=0.01 ), init_guess, method='SLSQP', bounds=bounds, constraints=constraints)
    else:
        result = minimize(lambda x: calculate_portfolio_volatility(returns, x ), init_guess, method='SLSQP', bounds=bounds, constraints=constraints)
    
    # Chuyển đổi tỷ trọng tối ưu thành từ điển với tên cổ phiếu
    optimal_weights = dict(zip(returns.columns, result.x))
    
    return optimal_weights

In [27]:
portfolio_optimize(returns,sharpe_ratio_or_variance=True)

Degrees of freedom <= 0 for slice
divide by zero encountered in divide
invalid value encountered in multiply


{'AAA': 0.5, 'A32': 0.5}

In [10]:
df=get_api_portfolio(['AAM', 'XPH', 'YEG', 'AAT'], n=6)
print(df)
# symbols=['AAA', 'A32']

# predictions = pd.DataFrame(index=data.index, columns=symbols)
# predictions

{'error': 'Error for symbol XPH: Reindexing only valid with uniquely valued Index objects'}


In [6]:
# # Giả sử df là DataFrame chứa dữ liệu của bạn
# # Chuyển đổi idx_time thành kiểu chuỗi với định dạng tháng-năm
# df['idx_time_str'] = df['idx_time'].astype(str) + '-01'

# # Chuyển đổi chuỗi thành kiểu datetime
# df['idx_time'] = pd.to_datetime(df['idx_time_str'], format='%Y-%m-%d')

# # Tính số ngày kể từ mốc thời gian đầu tiên
# df['days'] = (df['idx_time'] - df['idx_time'].min()).dt.days

# # Lọc dữ liệu không bị thiếu
# df = df.dropna(subset=['AAA', 'A32'])

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime

def predict_future_prices(symbols, n):
    """
    Dự đoán giá cổ phiếu trong tương lai dựa trên dữ liệu đầu vào.
    
    Parameters:
    - data (list of dict): Danh sách các từ điển chứa dữ liệu với cột 'idx_time', 'AAA', và 'A32'.
    - n (int): Số tháng từ ngày hiện tại để dự đoán.
    
    Returns:
    - future_df (pd.DataFrame): DataFrame chứa dự đoán giá cổ phiếu trong tương lai.
    """
    data=get_api_portfolio(symbols)

    try:
        df = pd.DataFrame(eval(data))
    except ValueError as e:
        print(f"Error creating DataFrame: {e}")
        return
    
    if df.empty or not {'idx_time', 'AAA', 'A32'}.issubset(df.columns):
        print("Data không hợp lệ. Đảm bảo dữ liệu chứa các cột 'idx_time', 'AAA', và 'A32'.")
        return
    
    df['idx_time'] = pd.to_datetime(df['idx_time'] + '-01', format='%Y-%m-%d')
    
    df['days'] = (df['idx_time'] - df['idx_time'].min()).dt.days
    
    df = df.dropna(subset=['AAA', 'A32'])
    
    degree = 2
    
    coeffs_AAA = np.polyfit(df['days'], df['AAA'], degree)
    coeffs_A32 = np.polyfit(df['days'], df['A32'], degree)
    
    # Tạo dãy tháng trong tương lai
    future_dates = pd.date_range(start=pd.to_datetime('today').replace(day=1), periods=n, freq='MS')
    future_days = (future_dates - df['idx_time'].min()).days
    
    # Dự đoán giá cho cả hai cổ phiếu
    predicted_AAA = np.polyval(coeffs_AAA, future_days)
    predicted_A32 = np.polyval(coeffs_A32, future_days)
    
    # Tạo DataFrame cho kết quả dự đoán
    future_df = pd.DataFrame({
        'idx_time': future_dates.strftime('%Y-%m'),  # Giữ định dạng tháng-năm
        'AAA': predicted_AAA,
        'A32': predicted_A32,
    })
    last_prediction = future_df.loc[len(future_df)-1]
    # print(last_prediction)
    result = {
        # 'idx_time': last_prediction['idx_time'],
        'AAA': last_prediction['AAA'],
        'A32': last_prediction['A32']
    }
    
    return json.dumps(result, default=str)


In [10]:
def predict_future_prices(symbols, n):
    """
    Dự đoán giá cổ phiếu trong tương lai dựa trên dữ liệu đầu vào. Phù hợp cho việc dự báo tình hình của danh mục 

    Args:
        symbols: Danh sách mã cổ phiếu cần dự đoán.
        n: Số tháng từ ngày hiện tại để dự đoán.
    """
    data = get_api_portfolio(symbols)

    try:
        df = pd.DataFrame(eval(data))
    except (ValueError, SyntaxError) as e:
        print(f"Error creating DataFrame: {e}")
        return
    
    if df.empty or not {'idx_time', 'AAA', 'A32'}.issubset(df.columns):
        print("Data không hợp lệ. Đảm bảo dữ liệu chứa các cột 'idx_time', 'AAA', và 'A32'.")
        return
    
    df['idx_time'] = pd.to_datetime(df['idx_time'] + '-01', format='%Y-%m-%d')
    
    df['days'] = (df['idx_time'] - df['idx_time'].min()).dt.days
    
    df = df.dropna(subset=symbols)
    
    degree = 2
    coeffs = {}

    for symbol in symbols:
        coeffs[symbol] = np.polyfit(df['days'], df[symbol], degree)
    
    # Tạo dãy tháng trong tương lai
    future_dates = pd.date_range(start=pd.to_datetime('today').replace(day=1), periods=n, freq='MS')
    future_days = (future_dates - df['idx_time'].min()).days
    
    # Dự đoán giá cho các cổ phiếu
    predictions = {}
    for symbol in symbols:
        predictions[symbol] = np.polyval(coeffs[symbol], future_days)
    
    # Tạo DataFrame cho kết quả dự đoán
    future_df = pd.DataFrame({
        'idx_time': future_dates.strftime('%Y-%m'),  # Giữ định dạng tháng-năm
        **predictions
    })
    
    last_prediction = future_df.iloc[-1]
    
    result = {
        # 'idx_time': last_prediction['idx_time'],
        **{symbol: last_prediction[symbol] for symbol in symbols}
    }
    
    return json.dumps(result, default=str)

In [11]:
future_predictions = predict_future_prices(symbols=['AAA', 'A32'], n=6)
print(future_predictions)

{"AAA": -0.391964093313804, "A32": -0.011103280506613383}


In [170]:
from scipy.optimize import minimize

def port_sharpe(weights, returns):
    """
    Tính toán tỷ lệ Sharpe của danh mục đầu tư.
    
    Parameters:
    - weights (np.array): Tỷ trọng của các cổ phiếu trong danh mục đầu tư.
    - returns (pd.DataFrame): DataFrame chứa lợi suất của các cổ phiếu.
    
    Returns:
    - sharpe_ratio (float): Tỷ lệ Sharpe của danh mục đầu tư.
    """
    port_return = np.sum(returns.mean() * weights)
    port_stddev = np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights)))
    return port_return / port_stddev

def port_variance(weights, returns):
    """
    Tính toán biến động của danh mục đầu tư.
    
    Parameters:
    - weights (np.array): Tỷ trọng của các cổ phiếu trong danh mục đầu tư.
    - returns (pd.DataFrame): DataFrame chứa lợi suất của các cổ phiếu.
    
    Returns:
    - variance (float): Biến động của danh mục đầu tư.
    """
    return np.dot(weights.T, np.dot(returns.cov(), weights))

def portfolio_optimize(returns, sharpe_ratio_or_variance=True):
    """
    Tối ưu hóa danh mục đầu tư dựa trên tỷ lệ Sharpe hoặc biến động.
    
    Parameters:
    - returns (pd.DataFrame): DataFrame chứa lợi suất của các cổ phiếu.
    - sharpe_ratio_or_variance (bool): Nếu True, tối ưu hóa theo tỷ lệ Sharpe; nếu False, tối ưu hóa theo biến động.
    
    Returns:
    - optimal_weights (dict): Tỷ trọng tối ưu của các cổ phiếu trong danh mục đầu tư dưới dạng từ điển.
    """
    returns = json.loads(returns)
    returns = pd.DataFrame([returns])
    # returns=returns[returns.columns[1:]]
    # print(returns)
    num_assets = len(returns.columns)
    # print('haha',num_assets)
    init_guess = num_assets * [1. / num_assets]
    bounds = tuple((0, 1) for asset in range(num_assets))
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    if sharpe_ratio_or_variance:
        result = minimize(lambda x: -port_sharpe(x, returns), init_guess, method='SLSQP', bounds=bounds, constraints=constraints)
    else:
        result = minimize(lambda x: port_variance(x, returns), init_guess, method='SLSQP', bounds=bounds, constraints=constraints)
    
    # Chuyển đổi tỷ trọng tối ưu thành từ điển với tên cổ phiếu
    optimal_weights = dict(zip(returns.columns, result.x))
    
    return optimal_weights


optimal_weights = portfolio_optimize(future_predictions, sharpe_ratio_or_variance=True)
optimal_weights

{'AAA': 0.5, 'A32': 0.5}

In [110]:
# Chọn mức độ của hồi quy polynomial
degree = 2

# Tạo mô hình hồi quy cho cả hai cổ phiếu
coeffs_AAA = np.polyfit(df['days'], df['AAA'], degree)
coeffs_A32 = np.polyfit(df['days'], df['A32'], degree)

In [111]:
future_dates = pd.date_range(start='2024-08-01', end='2030-12-31', freq='MS')  # MS: bắt đầu của tháng
future_days = (future_dates - df['idx_time'].min()).days

# Dự đoán giá cho cả hai cổ phiếu
predicted_AAA = np.polyval(coeffs_AAA, future_days)
predicted_A32 = np.polyval(coeffs_A32, future_days)

# Tạo DataFrame cho kết quả dự đoán
future_df = pd.DataFrame({
    'idx_time': future_dates.strftime('%Y-%m'),  # Giữ định dạng tháng-năm
    'AAA': predicted_AAA,
    'A32': predicted_A32,
})
future_df

,idx_time,AAA,A32
0,2024-08,-0.061434,0.003340
1,2024-09,-0.108756,0.000838
2,2024-10,-0.162597,-0.001368
3,2024-11,-0.226546,-0.003424
4,2024-12,-0.296478,-0.005198
...,...,...,...
72,2030-08,-24.211449,0.385307
73,2030-09,-24.855963,0.398845
74,2030-10,-25.487731,0.412163
75,2030-11,-26.148872,0.426148


In [97]:
symbols=['AAA', 'A32']
predictions = pd.DataFrame(index=data.index, columns=symbols)
for symbol in symbols:
    prices = data[symbol].values
    dates = np.arange(len(prices))  # Chỉ số ngày

    # Hồi quy tuyến tính
    coeffs = np.polyfit(dates, prices, 1)
    poly = np.poly1d(coeffs)

    # Dự đoán giá tương lai trong 30 ngày
    future_dates = np.arange(len(prices), len(prices) + 30)
    future_predictions = poly(future_dates)

    # Lưu dự đoán vào DataFrame
    predictions[symbol] = np.concatenate([prices, future_predictions])

result = predictions.reset_index().to_dict(orient='records')

ValueError: Length of values (42) does not match length of index (12)

In [ ]:
n_months = 12
predictions = []

# Dự đoán tháng đầu tiên
with torch.no_grad():
    prediction = model(input_tensor)

# Lưu dự đoán và chuẩn bị đầu vào cho lần dự đoán tiếp theo
predictions.append(prediction.squeeze(0).numpy())  # Xóa batch dimension

# Dự đoán liên tiếp
for _ in range(n_months - 1):
    # Lấy dự đoán cuối cùng để làm đầu vào cho lần dự đoán tiếp theo
    next_input = prediction[:, -1, :]  # Lấy dự đoán cuối cùng
    input_tensor = torch.cat((input_tensor[:, 1:, :], next_input.unsqueeze(0)), dim=1)
    
    # Dự đoán tháng tiếp theo
    with torch.no_grad():
        prediction = model(input_tensor)
    
    # Lưu dự đoán
    predictions.append(prediction.squeeze(0).numpy())

# Chuyển đổi dự đoán thành DataFrame
predictions_df = pd.DataFrame(np.concatenate(predictions, axis=0), columns=['AAA', 'A32'])
predictions_df.index = pd.date_range(start=df['idx_time'].max() + pd.DateOffset(months=1), periods=n_months, freq='M')

print(predictions_df)

In [14]:
endday = datetime.now()
startday = endday - timedelta(days=365)
startday.strftime('%Y-%m-%d')

'2023-08-26'

In [36]:
data = {'timestamp': ['2023-01-15', '2023-02-20', '2024-03-25']}
df = pd.DataFrame(data)

# Chuyển cột 'timestamp' thành kiểu datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Trích xuất tháng và năm từ cột 'timestamp'
df['month'] = df['timestamp'].dt.month
df['year'] = df['timestamp'].dt.year

# Nếu bạn muốn kết hợp tháng và năm thành một cột duy nhất
df['month_year'] = df['timestamp'].dt.to_period('M')

print(df)

   timestamp  month  year month_year
0 2023-01-15      1  2023    2023-01
1 2023-02-20      2  2023    2023-02
2 2024-03-25      3  2024    2024-03
